In [1]:
from ase import Atoms
import os
import string
import random
from pymatgen.io.cif import CifParser
from urllib.request import urlopen
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ase.build import bulk, niggli_reduce
from gpaw import GPAW, PW, FermiDirac
from ase.io import read
import pandas as pd

import glob

# Importing the gpaw pseudo

In [5]:
folder = './'
g = glob.glob(
    folder + 'gpaw-setups-0.9.20000/*.PBE.gz')

# Checking the available atoms on gpaw

In [6]:
available_atoms = []
for i in g:
    available_atoms += [i.replace(
        folder + 'gpaw-setups-0.9.20000/', '').split('.')[0]]
    
print(available_atoms[:10])

['Cd', 'Ga', 'Ar', 'Br', 'Y', 'Ta', 'Cl', 'Pt', 'Ag', 'Mn']


# Function to check if the atoms is in pseudo

In [7]:
def fix_psuedo(a):
    for i in range(len(a)):
        if not a[i].symbol in available_atoms:
            print(a[i].symbol, 'is not available in GPAW, replacing it with Y')
            a[i].symbol = 'Y'

# Importing database

In [9]:
# Specify the path to the Excel file
excel_file_path = 'DataBaseV2.xlsx'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path)

df.head()

,formula_pretty,formula_anonymous,chemsys,material_id,structure,formation_energy_per_atom,is_metal,e_total,e_ionic,e_electronic,n,fields_not_requested,structure_cif
0,YCrB4,ABC4,B-Cr-Y,mp-20450,Full Formula (Y4 Cr4 B16)\nReduced Formula: YC...,-0.653251,False,47.348925,5.890129,41.458796,6.438851,"['builder_meta', 'nsites', 'elements', 'neleme...",# generated using pymatgen\ndata_YCrB4\n_symme...
1,LiCaGaF6,ABCD6,Ca-F-Ga-Li,mp-12829,Full Formula (Li2 Ca2 Ga2 F12)\nReduced Formul...,-3.463849,False,6.967909,4.827025,2.140884,1.463176,"['builder_meta', 'nsites', 'elements', 'neleme...",# generated using pymatgen\ndata_LiCaGaF6\n_sy...
2,Na2GeO3,AB2C3,Ge-Na-O,mp-5784,Full Formula (Na4 Ge2 O6)\nReduced Formula: Na...,-2.140016,False,11.718332,8.805022,2.913310,1.706842,"['builder_meta', 'nsites', 'elements', 'neleme...",# generated using pymatgen\ndata_Na2GeO3\n_sym...
3,CsMgCl3,ABC3,Cl-Cs-Mg,mp-23004,Full Formula (Cs2 Mg2 Cl6)\nReduced Formula: C...,-2.322267,False,10.490509,7.869904,2.620605,1.618828,"['builder_meta', 'nsites', 'elements', 'neleme...",# generated using pymatgen\ndata_CsMgCl3\n_sym...
4,Li2VSiO5,ABC2D5,Li-O-Si-V,mp-18860,Full Formula (Li8 V4 Si4 O20)\nReduced Formula...,-2.766672,False,16.969145,13.458970,3.510175,1.873546,"['builder_meta', 'nsites', 'elements', 'neleme...",# generated using pymatgen\ndata_Li2VSiO5\n_sy...


In [10]:
df.shape

(7290, 13)

# Using DFT based descriptors

In [ ]:
maxatom = 1000 # to limit the number of atoms

for num, cif in enumerate(np.array(df['structure_cif'])[:maxatom]):
    output = {}
    # print(f"Material {num}/{len(np.array(df['structure_cif']))}")
    print(f"Material {num+1}/{len(np.array(df['structure_cif'])[:maxatom])}")
    print("-"*50)
    try:
        parser = CifParser.from_string(cif)
        structure = parser.get_structures()
        structure = structure[0]
        atoms = Atoms(pbc=True, cell=structure.lattice.matrix,
                    positions=structure.cart_coords, numbers=structure.atomic_numbers)

        calculation_type = 'bulk'

        if calculation_type == 'bulk':
            niggli_reduce(atoms)

        # Perform any necessary modifications to the structure
        fix_psuedo(atoms)

        # Perform the GPAW calculation
        calc = GPAW(mode='lcao',
                    xc='PBE',
                    convergence={'density': 1.e-6},
                    txt='output.txt')

        calc.calculate(atoms)
        H = calc.hamiltonian
        ef = calc.get_fermi_level()
        num_atoms = atoms.get_global_number_of_atoms()


        # Update the output dictionary with the results
        output['e_band'] = H.e_band
        output['e_coulomb'] = H.e_coulomb
        output['e_entropy'] = H.e_entropy
        output['e_external'] = H.e_external
        output['e_kinetic'] = H.e_kinetic
        output['e_kinetic0'] = H.e_kinetic0
        output['e_xc'] = H.e_xc
        output['e_total_free'] = H.e_total_free
        output['ef'] = ef
        

        output['e_band/num_atoms'] = H.e_band/num_atoms
        output['e_coulomb/num_atoms'] = H.e_coulomb/num_atoms
        output['e_entropy/num_atoms'] = H.e_entropy/num_atoms
        output['e_external/num_atoms'] = H.e_external/num_atoms
        output['e_kinetic/num_atoms'] = H.e_kinetic/num_atoms
        output['e_kinetic0/num_atoms'] = H.e_kinetic0/num_atoms
        output['e_xc/num_atoms'] = H.e_xc/num_atoms
        output['e_total_free/num_atoms'] = H.e_total_free/num_atoms

        output_list.append(output)
        
    except Exception as e:
            print('Problem in GPAW')
            print(f"index {num} is skipped")
            skiped_index.append(num)
            print(f"Number of skipped material: {len(skiped_index)}")
            print("-"*50)
            

df = df.drop(skiped_index)
df_cif = pd.DataFrame(output_list)
df = pd.concat([df,df_cif], axis=1)
df = df.dropna()

# Specify the path where you want to save the Excel file
excel_file_path = 'output.xlsx'

# Export the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print(f'DataFrame has been exported to {excel_file_path}')